---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
import warnings
warnings.filterwarnings('ignore')

#from sklearn.preprocessing import MinMaxScaler


#['B', 'C'], axis=1
#df.drop(['B', 'C'], axis=1)


def blight_model():
    train_df = pd.read_csv("train.csv", encoding = 'ISO-8859-1', low_memory=False)
    test_df = pd.read_csv("test.csv") # Might be an encoding problem
    
    #dropping values with NaN compliance
    train_df = train_df[np.isfinite(train_df['compliance'])]
    
    ###
    ### Converting STR data to INT data, working with text data
    ###
    
    ### CONVERTING "agency_name" DATA TO INT LABELS
    agency_label = train_df["agency_name"].append(test_df["agency_name"], ignore_index=True)
    for a,b in enumerate(agency_label.unique()):
        train_df.replace(b, a, inplace=True)
        test_df.replace(b, a, inplace=True)
        
    ### CONVERTING "inspector_name" to str
    train_df["inspector_name"] = train_df["inspector_name"].str.len()
    test_df["inspector_name"] = test_df["inspector_name"].str.len()  
    
    ### CHECKING "violator_name". IF FUNNY NAME IN "violator_name" RETURN 1 ELSE RETURN 0
    train_df["violator_name_len"] = train_df["violator_name"].str.len()
    test_df["violator_name_len"] = test_df["violator_name"].str.len()
    
    #train_df = train_df[(train_df[["violator_name_len"]] > 1).all(axis=1)]
    #test_df = test_df[(test_df[["violator_name_len"]] > 1).all(axis=1)]
    
    funny_names = ["L.L.C.", "LLC", "LTD","INC", "CO", "CORPORATION", "CORP", "GROUP", "HOLDING"]
    violator_name_bin = []
    for b in (train_df["violator_name"]):
        try:
            if any(name in b for name in funny_names):
                violator_name_bin.append(1)
            else:
                violator_name_bin.append(0) 
        except:
            violator_name_bin.append(0)
    train_df["violator_name"] = pd.Series(data=violator_name_bin)

    violator_name_bin = []
    for b in (test_df["violator_name"]):
        try:
            if any(name in b for name in funny_names):
                violator_name_bin.append(1)
            else:
                violator_name_bin.append(0)
        except:
            violator_name_bin.append(0)    
    test_df["violator_name"] = pd.Series(data=violator_name_bin)
    
    #DROPPING violation_street_name, violation_zip_code, mailing_address_str_name 
    train_df = train_df.drop(["violator_name_len", "violation_street_name", "violation_zip_code", "mailing_address_str_name"], axis=1)
    test_df = test_df.drop(["violator_name_len","violation_street_name", "violation_zip_code", "mailing_address_str_name"], axis=1)
    
    ###CHECKING IF violation_street_number is equal to mailing_address_str_number 
    train_df['violation_street_number'] = pd.to_numeric(train_df['violation_street_number'], errors='coerce').fillna(0).astype(int)
    train_df['mailing_address_str_number'] = pd.to_numeric(train_df['mailing_address_str_number'], errors='coerce').fillna(0).astype(int)
    test_df['violation_street_number'] = pd.to_numeric(test_df['violation_street_number'], errors='coerce').fillna(0).astype(int)
    test_df['mailing_address_str_number'] = pd.to_numeric(test_df['mailing_address_str_number'], errors='coerce').fillna(0).astype(int)
    
    train_df['violation_street_number'] = train_df['violation_street_number']/train_df['mailing_address_str_number']
    train_df['violation_street_number'] = np.where(train_df['violation_street_number'] == 1, 1,0)
    train_df.rename(columns = {'violation_street_number': 'violation_st_num_is_eqial_mailing_st_num'})
     
    test_df['violation_street_number'] = test_df['violation_street_number']/train_df['mailing_address_str_number']
    test_df['violation_street_number'] = np.where(test_df['violation_street_number'] == 1, 1,0)
    test_df.rename(columns = {'violation_street_number': 'violation_st_num_is_eqial_mailing_st_num'})
    
    train_df = train_df.drop(["mailing_address_str_number"], axis=1)
    test_df = test_df.drop(["mailing_address_str_number"], axis=1)
    
    ### DROPPING non_us_str_code, country
    
    train_df = train_df.drop(["non_us_str_code", "country"], axis=1)
    test_df = test_df.drop(["non_us_str_code", "country"], axis=1)
    
    ### DROPPING СOLUMNS non_us_str_code, country
    
    train_df = train_df.drop(["payment_amount", "balance_due", "payment_date", 
                              "payment_status", "collection_status", "compliance_detail"], axis=1)
    
    ### DROPPING city, zip_code, violation_description, grafitti_status CAUSE I FOUND THEM USELESS
    train_df = train_df.drop(["city", "zip_code", "violation_description", "grafitti_status"], axis=1)
    test_df = test_df.drop(["city", "zip_code", "violation_description", "grafitti_status"], axis=1)
    
    ### CONVERTING state to INT: #factorize?
    for a,b in enumerate(train_df['state'].unique()):
        train_df['state'].replace(b, a, inplace=True)
        test_df['state'].replace(b, a, inplace=True)
    
    ### CONVERTING violation_code to INT:
    c = train_df['violation_code'].append(test_df['violation_code'], ignore_index=True)
    for a,b in enumerate(c.unique()):
        train_df['violation_code'].replace(b, a, inplace=True)
        test_df['violation_code'].replace(b, a, inplace=True)
        
    ### CONVERTING disposition to INT:
    c = train_df['disposition'].append(test_df['disposition'], ignore_index=True)
    for a,b in enumerate(c.unique()):
        train_df['disposition'].replace(b, a, inplace=True)
        test_df['disposition'].replace(b, a, inplace=True)
    
    ### PICKING RIGHT PART OF STR TO CONVERT
    train_df['ticket_issued_date'] = train_df['ticket_issued_date'].str[:10]
    test_df['ticket_issued_date'] = test_df['ticket_issued_date'].str[:10]
    train_df['hearing_date'] = train_df['hearing_date'].str[:10]
    test_df['hearing_date'] = test_df['hearing_date'].str[:10]    
    
    ### CONVERTING dtype = object to datetime64[ns]
    train_df['ticket_issued_date'] = pd.to_datetime(train_df['ticket_issued_date'])
    test_df['ticket_issued_date'] = pd.to_datetime(test_df['ticket_issued_date'])
    train_df['hearing_date'] = pd.to_datetime(train_df['hearing_date'])
    test_df['hearing_date'] = pd.to_datetime(test_df['hearing_date'])
    
    ### CONVERTING DATA TO DAYS
    train_df['ticket_issued_date'] = pd.to_timedelta(train_df['ticket_issued_date'])
    test_df['ticket_issued_date'] = pd.to_timedelta(test_df['ticket_issued_date'])
    train_df['hearing_date'] = pd.to_timedelta(train_df['hearing_date'])
    test_df['hearing_date'] = pd.to_timedelta(test_df['hearing_date'])
    
    train_df['ticket_issued_date'] = (train_df['hearing_date'] - train_df['ticket_issued_date'])
    test_df['ticket_issued_date'] = (test_df['hearing_date'] - test_df['ticket_issued_date'])
    
    train_df['ticket_issued_date']= (train_df['ticket_issued_date']).astype('timedelta64[D]')
    test_df['ticket_issued_date']= (test_df['ticket_issued_date']).astype('timedelta64[D]')
    
    ### CALCULATING DELTA BETWEEN ticket hearing date and ticked issued data
    train_df = train_df.rename(columns = {'ticket_issued_date':'hearing_delta'})
    test_df = test_df.rename(columns = {'ticket_issued_date':'hearing_delta'})
    
    train_df = train_df.drop(['hearing_date'], axis=1) 
    test_df = test_df.drop(['hearing_date'], axis=1) 
        
    ###    
    ### CHECKING DTYPES AND FILLING NA
    ###
    
    train_df.replace([np.inf, -np.inf], np.nan)
    test_df.replace([np.inf, -np.inf], np.nan)
    
    train_df.fillna(method='pad', inplace = True)
    test_df.fillna(method='pad', inplace = True)
    
    #data_types = pd.concat([train_df.dtypes, test_df.dtypes], axis=1, keys=["train_df","test_df"], sort=False)
    
    ### SPLITTING DATA
    X_train = train_df.drop(["compliance"], axis=1)
    y_train = train_df["compliance"]
    X_test = test_df
    #y_test = 
    
    ###
    ### ADDING addresses.csv + latlons.csv and combining them with train and test data
    ###
    
    addresses = pd.read_csv("addresses.csv")
    latlons = pd.read_csv("latlons.csv")
    
    #print(addresses.dtypes)
    #print(latlons.dtypes)
 
    output = pd.merge(latlons, addresses, how='inner', on=['address'])
    output = output.drop(['address'],axis = 1)
    
    X_train = pd.merge(X_train, output, how='left', on=['ticket_id'])
    X_test = pd.merge(X_test, output, how='left', on=['ticket_id'])
    
    X_train.fillna(method='pad', inplace = True)
    X_test.fillna(method='pad', inplace = True) #is pad the best here? I would simply not use it in case if the value is NAN
 
    ###
    ### INITIALIZING ML ALGORITHMS
    ###
    
    #scaler = MinMaxScaler()
    #X_train_scaled = scaler.fit_transform(X_train)
    #X_test_scaled = scaler.transform(X_test)
    
    grid_values = {'penalty': ['l1','l2'], 'C': [0.001,0.01,0.1,1,10,100,1000]}
    clf = LogisticRegression()
    clf_wGridSearch = GridSearchCV(clf, param_grid = grid_values)
    clf_wGridSearch = clf_wGridSearch.fit(X_train, y_train)
    
    #proba = cross_val_predict(clf, X, y, cv=cv, method='predict_proba')
    
    y_test_proba = clf_wGridSearch.predict_proba(X_test)
    #print('Accuracy of classifier on training set: {:.2f}'.format(clf.score(X_train, y_train)))

    #return X_train
    #return output
    return pd.Series(y_test_proba[:,1], index=X_test["ticket_id"])
blight_model()

ticket_id
284932    0.065753
285362    0.020371
285361    0.069691
285338    0.039688
285346    0.069105
285345    0.038739
285347    0.071855
285342    0.120207
285530    0.016267
284989    0.030655
285344    0.071367
285343    0.020577
285340    0.020577
285341    0.071366
285349    0.063893
285348    0.035730
284991    0.024786
285532    0.030677
285406    0.029933
285001    0.035554
285006    0.014109
285405    0.019557
285337    0.023869
285496    0.070139
285497    0.038221
285378    0.020235
285589    0.024369
285585    0.037540
285501    0.070482
285581    0.020240
            ...   
376367    0.016854
376366    0.042292
376362    0.099381
376363    0.064771
376365    0.016854
376364    0.042292
376228    0.044129
376265    0.043270
376286    0.295760
376320    0.043649
376314    0.042814
376327    0.282534
376385    0.287084
376435    0.135468
376370    0.143475
376434    0.092876
376459    0.079510
376478    0.001776
376473    0.043425
376484    0.037607
376482    0.017670
37

In [2]:
bm = blight_model()
res = 'Data type Test: '
res += ['Failed: type(bm) should Series\n','Passed\n'][type(bm)==pd.Series]
res += 'Data shape Test: '
res += ['Failed: len(bm) should be 61001\n','Passed\n'][len(bm)==61001]
res += 'Data Values Test: '
res += ['Failed: all values should be in [0.,1.]\n','Passed\n'][all((bm<=1.) & (bm>=0.))]
res += 'Data Values type Test: '
res += ['Failed: bm.dtype should be float\n','Passed\n'][str(bm.dtype).count('float')>0]
res += 'Index type Test: '
res += ['Failed: type(bm.index) should be Int64Index\n','Passed\n'][type(bm.index)==pd.Int64Index]
res += 'Index values type Test: '
res += ['Failed: type(bm.index[0]) should be int64\n','Passed\n'][str(type(bm.index[0])).count("int64")>0]

res += 'Output index shape test:'
res += ['Failed, bm.index.shape should be (61001,)\n','Passed\n'][bm.index.shape==(61001,)]

res += 'Output index test: '
if bm.index.shape==(61001,):
    res +=['Failed\n','Passed\n'][all(pd.read_csv('test.csv',usecols=[0],index_col=0).sort_index().index.values==bm.sort_index().index.values)]
else:
    res+='Failed'
print(res)

Data type Test: Passed
Data shape Test: Passed
Data Values Test: Passed
Data Values type Test: Passed
Index type Test: Passed
Index values type Test: Passed
Output index shape test:Passed
Output index test: Passed

